<a href="https://colab.research.google.com/github/mybox-lab/de_test16/blob/main/16_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=06283cacc9a84eb6d78095e005dfe29d85ed4b6c1b3007a5af412e1cc09e7f1a
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [11]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql import SparkSession
import random
import datetime
import string

def generate_random_name():
    first_names = ["Иван", "Алексей", "Мария", "Анна"]
    last_names = ["Иванов", "Петров", "Смирнов", "Сидоров"]
    return random.choice(first_names) + ' ' + random.choice(last_names)

def generate_random_email(name, age):
    domains = ["gmail.com", "yahoo.com", "outlook.com"]
    return f"{name.lower().replace(' ', '_')}{age}@{random.choice(domains)}"

def generate_random_city():
    cities = ["Москва", "Санкт-Петербург", "Новосибирск", "Екатеринбург", "Казань"]
    return random.choice(cities)

def generate_registration_date(age):
    today = datetime.date.today()
    return today - datetime.timedelta(days=365 * age)

def generate_data(num_rows):
    schema = StructType([
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("email", StringType(), True),
        StructField("city", StringType(), True),
        StructField("age", IntegerType(), True),
        StructField("salary", IntegerType(), True),
        StructField("registration_date", DateType(), True)
    ])

    data = [
        (i, name, generate_random_email(name, age), generate_random_city(), age, random.randint(30000, 150000), generate_registration_date(age))
        for i, name, age in zip(range(1, num_rows+1), [generate_random_name() for _ in range(num_rows)], [random.randint(18, 95) for _ in range(num_rows)])
    ]

    df = spark.createDataFrame(data, schema)

    # Введение NULL значений
    df = df.withColumn("salary", F.when(F.rand() < 0.05, F.lit(None)).otherwise(col("salary")))

    # Сохранение в CSV
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")
    file_name = f"{current_date}-dev.csv"
    df.write.mode("overwrite").csv(file_name, header=True)

if __name__ == "__main__":
    spark = SparkSession.builder.appName("DataGenerator").getOrCreate()
    num_rows = int(input("Введите количество строк: "))
    generate_data(num_rows)
    spark.stop()

Введите количество строк: 10
